In [3]:
# To find values within a column...
#df.loc[df['column_name'] == some_value]
#x = dfs[0][1]
#x.loc[x["ncli"] == 8151]

In [4]:
import os
import re
import math
import pandas as pd
import numpy as np

import plotly
import plotly.plotly as py
from plotly.graph_objs import *

plotly.tools.set_credentials_file(username='theano', api_key='hkYRYxjyM0aAGA5nmGq1')

In [5]:
import matplotlib.pyplot as plt

import matplotlib
matplotlib.style.use('ggplot')

In [6]:
home = os.getcwd()
print(home)
sorted([x for x in os.listdir(home)])
collectl_stats = ['Date', '[CPU]User%', '[CPU]Nice%', '[CPU]Sys%', '[CPU]Wait%',
                               '[CPU]Irq%', '[CPU]Soft%', '[CPU]Steal%', '[CPU]Idle%',
                               '[CPU]Totl%', '[CPU]Intrpt/sec', '[CPU]Ctx/sec', '[CPU]Proc/sec',
                               '[CPU]ProcQue', '[CPU]ProcRun', '[CPU]L-Avg1', '[CPU]L-Avg5',
                               '[CPU]L-Avg15', '[CPU]RunTot', '[CPU]BlkTot']
loaded_files = []


/home/theano/mountpoint/tsch/mininetpipe/results/2017-12-07-15-01


In [7]:
def load_datfile(fpath, proto):
    # Returns a dataframe
    df = pd.read_csv(fpath, 
                     index_col=False,
                     names=["ncli",proto])
    return df


def load_collectl(fname):
    df = pd.read_csv(fname,
                     delim_whitespace=True,
                     comment="#",
                     names=collectl_stats)
    return df

def load_data_and_stats(basename):
    pds = []
    for fname in [x for x in os.listdir(".") if 
                  ("server-0" not in x and basename in x)]:
        df_tput = load_datfile(os.path.join(home, fname), basename)
        
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
            
        newdf = pd.concat([df_tput.sum(), df_stats.mean()])
        newdf["fname"] = fname
        pds.append(newdf)
    return (basename, pd.concat(pds, axis=1).transpose())

def load_data_no_sum(basename):
    pds = []
    for fname in [x for x in os.listdir(".") if 
                  ("server-0" not in x and basename in x)]:
        df_tput = load_datfile(os.path.join(home, fname), basename)
                
        collectl_fname = [x for x in os.listdir(".") if "%s-server-0" % fname in x]
        if len(collectl_fname) > 0:
            df_stats = load_collectl(collectl_fname[0])
        
        newdf = pd.concat([df_tput, df_stats])
        newdf["fname"] = fname
        pds.append(newdf)

    return (basename, pd.concat(pds))

In [8]:
df = load_data_and_stats("tcp_collect")

dfs = [df]
groups = [(name, df.drop("fname", axis=1).astype(float).groupby(["ncli"])) for name, df in dfs]
agged = [(name, gp.mean(), gp.std()) for name, gp in groups]
[x.size() for name, x in groups]

[ncli
 64.0       31
 128.0      30
 256.0      30
 512.0      30
 1024.0     36
 2048.0     30
 4096.0     35
 8192.0     38
 16384.0    31
 26824.0     1
 32764.0     1
 32768.0     3
 dtype: int64]

In [10]:
traces = []
barwidth_scalar = 1/4

for (name, means, standards) in agged:
    barwidth = means.index.values * barwidth_scalar
    #barwidth = 10
    
    # Collectl data
    traces.append(Bar(
        x = means.index.values,
        y = means["[CPU]Sys%"],
        yaxis = 'y2',
        name = '%s CPU Sys%%' % name,
        width = barwidth,
        opacity=0.6
    ))
    traces.append(Bar(
        x = means.index.values,
        y = means["[CPU]Soft%"],
        yaxis = 'y2',
        name = '%s CPU Soft%%' % name,
        width = barwidth,
        opacity=0.6
    ))
    traces.append(Bar(
        x = means.index.values,
        y = means["[CPU]User%"],
        yaxis = 'y2',
        name = '%s CPU User%%' % name,
        width = barwidth,
        opacity=0.6
    ))
    
    # Throughput
    traces.append(Scatter(
        x = means.index.values,
        y = means[name],
        name = name,
        error_y = dict(
            type='data',
            array=standards[name],
            visible=True
        ),
    ))
    
    
layout = Layout (
    xaxis = {'title': 'number of clients', 'type': 'log'},
    yaxis = {'title': 'throughput (pps)', 'rangemode': 'tozero'},
    yaxis2 = {'title': 'CPU utilization', 
              'side': 'right', 
              'overlaying': 'y',
              'showgrid': False},
    title = 'Small-packet throughput',
    legend = {'x': 0.01, 'y': 1},
    barmode = "stack"
)

fig = Figure(data=traces, layout=layout)

py.iplot(fig, filename='cpu-throughput')

In [ ]:
traces = []
barwidth_scalar = 1/4
idx = pd.IndexSlice
#barwidth = 2

# Collectl data
traces.append(Bar(
    y = nosum_groups_160_described.loc[:, idx[:, "mean"]][("[CPU]Totl%", "mean")],
    name = "[CPU]Totl%",
    yaxis = 'y2',
    #width = barwidth,
    opacity=0.6,
))

# Throughput
traces.append(Scatter(
    y = nosum_groups_160_described.loc[:, idx[:, "mean"]][("tcp_collect", "mean")],
    name = "throughput",
    legendgroup = "throughput",        
    error_y = dict(
        type='data',
        array=nosum_groups_160_described.loc[:, idx[:, "std"]][("tcp_collect", "std")],
        visible=True
    ),
    mode="markers",
))
    
    
layout = Layout (
    xaxis = {'title': 'individual experiments'},
    yaxis = {'title': 'throughput (pps)', 'rangemode': 'tozero'},
    yaxis2 = {'title': 'CPU utilization', 
              'side': 'right', 
              'overlaying': 'y',
              'showgrid': False},
    title = 'Cross-core variance in throughput over CPU utilization<br>CPU legend: blue=Sys, orange=User, green=Soft',
    legend = {'x': 0.01, 'y': 1},
    barmode = "stack",
    showlegend = True
)

fig = Figure(data=traces, layout=layout)

py.iplot(fig, filename='cpu-throughput')

In [11]:
data = []

i = 1
for label, df in groups[0][1]:
    data.append(Histogram(x=df["tcp_collect"],
                          nbinsx=20,
                          name="%ss" % label,
                          opacity=0.6,
                          histnorm="probability",
                          xaxis='x%d' % i,
                          yaxis='y%d' % i))
    i += 1
        
nrows = math.ceil(len(data)/2)
fig = plotly.tools.make_subplots(rows=nrows, cols=2)

for i, dat in enumerate(data):
    fig.append_trace(dat, math.ceil((i+1)/2), (i % 2) + 1)
    fig['layout']['xaxis%d' % (i + 1)].update(title='throughput (pps)')
    fig['layout']['yaxis%d' % (i + 1)].update(title='frequency')    
    
fig['layout'].update(title='Throughput frequency for varying experiment durations')
    
py.iplot(fig, filename="somefile")

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [12]:
data = []

i = 1
for label, df in nosum_gp:
    data.append(Histogram(x=df["tcp_collect"],
                          nbinsx=40,
                          name="%ss" % label,
                          opacity=0.6,
                          histnorm="probability",
                          xaxis='x%d' % i,
                          yaxis='y%d' % i))
    i += 1
        
nrows = math.ceil(len(data)/2)
fig = plotly.tools.make_subplots(rows=nrows, cols=2)

for i, dat in enumerate(data):
    fig.append_trace(dat, math.ceil((i+1)/2), (i % 2) + 1)
    fig['layout']['xaxis%d' % (i + 1)].update(title='throughput (pps)')
    fig['layout']['yaxis%d' % (i + 1)].update(title='frequency')    
    
fig['layout'].update(title='Per-process throughput frequency for varying experiment durations')
    
py.iplot(fig, filename="somefile")

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]

